In [1]:
import glob
from utils.analysis import Extrapolation

In [2]:
TAG = "STskim_nopresel"
babies = glob.glob(f"../analysis/studies/onelep/output_{TAG}/Run2/*.root")
data_babies = [baby for baby in babies if "data" in baby]
sig_babies = [baby for baby in babies if "VBSWH_mkW" in baby]
bkg_babies = list(set(babies) - set(data_babies + sig_babies))
print(data_babies)
print(sig_babies)
bkg_babies

['../analysis/studies/onelep/output_STskim_nopresel/Run2/data.root']
['../analysis/studies/onelep/output_STskim_nopresel/Run2/VBSWH_mkW.root']


['../analysis/studies/onelep/output_STskim_nopresel/Run2/TTX.root',
 '../analysis/studies/onelep/output_STskim_nopresel/Run2/Bosons.root',
 '../analysis/studies/onelep/output_STskim_nopresel/Run2/VH.root',
 '../analysis/studies/onelep/output_STskim_nopresel/Run2/SingleTop.root',
 '../analysis/studies/onelep/output_STskim_nopresel/Run2/TTbar1L.root',
 '../analysis/studies/onelep/output_STskim_nopresel/Run2/TTbar2L.root',
 '../analysis/studies/onelep/output_STskim_nopresel/Run2/WJets.root']

In [3]:
vbswh = Extrapolation(
    sig_root_files=sig_babies,
    bkg_root_files=bkg_babies,
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=["xsec_sf", "lep_sf"],
    plots_dir=f"/home/users/jguiang/public_html/onelep_plots/{TAG}/val"
)
vbswh.df["presel_noVBS_noBVeto"] = vbswh.df.eval(
    "hbbjet_score > 0.3"
)
vbswh.df["presel_noVBS"] = vbswh.df.eval(
    "passes_bveto and hbbjet_score > 0.3"
)
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)

In [4]:
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 1400 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
    label="$S_T$ [GeV]",
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 1400 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,10.702483337379968,0.8627783738194451,2.303751990644384,0.631021943020261,BLINDED,BLINDED
presel_noDetaJJ and M_jj > 600 and ST > 1400 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,16.995866030306885,0.9673105236765668,0.4602654836639976,0.295980464556029,14,3.7416573867739413
presel_noDetaJJ and M_jj > 600 and ST > 1400 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,10.396652601074155,0.7061966894790528,3.822115987995458,0.8096381069586766,BLINDED,BLINDED
presel_noDetaJJ and M_jj > 600 and ST > 1400 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,4.018459167344179,0.45821899079156553,117.25605028148982,4.625308192586667,BLINDED,BLINDED

name,extp,rel_err
AtoB,1.5880301323101687,0.09868129981932727
BtoC,0.

In [ ]:
is_ttbar_sample = "(name == 'TTbar1L' | name == 'TTbar2L' | name == 'TTX')"
vbswh.ABCD(
    "ST > 1200",
    "abs(deta_jj) > 4 and M_jj > 1500",
    f"passes_bveto and {is_ttbar_sample}",
    h_dir="left", v_dir="up", 
    label="$S_T$ [GeV]"
)

In [5]:
print(("sig", "bkg"))
print(vbswh.get_event_counts(selection="abs(deta_jj) > 4 and M_jj > 1500 and passes_bveto and ST > 1200"))
print(vbswh.get_event_counts(selection="abs(deta_jj) > 4 and M_jj > 1500 and passes_bveto and ST > 1200", raw=True))
print(vbswh.get_event_errors(selection="abs(deta_jj) > 4 and M_jj > 1500 and passes_bveto and ST > 1200"))

('sig', 'bkg')
(113.26783108711243, 3.341920200833556)
(759, 321)
(4.603505970226833, 0.3723100665151049)


In [6]:
print(("sig", "bkg"))
print(vbswh.get_event_counts(selection="abs(deta_jj) > 4 and M_jj > 1500 and not passes_bveto and ST > 1200"))
print(vbswh.get_event_counts(selection="abs(deta_jj) > 4 and M_jj > 1500 and not passes_bveto and ST > 1200", raw=True))
print(vbswh.get_event_errors(selection="abs(deta_jj) > 4 and M_jj > 1500 and not passes_bveto and ST > 1200"))

('sig', 'bkg')
(3.280091166496277, 7.988692310784245)
(25, 392)
(0.7355304827309638, 0.7595408539285631)


In [7]:
# vbswh.test_selections(
#     [
#         "abs(deta_jj) > 4 and M_jj > 1500 and passes_bveto and (not (ST > 1200))",
#         "abs(deta_jj) > 4 and M_jj > 1500 and (not (passes_bveto)) and (not (ST > 1200))",
#         "abs(deta_jj) > 4 and M_jj > 1500 and (not (passes_bveto)) and ST > 1200",
#         "abs(deta_jj) > 4 and M_jj > 1500 and passes_bveto and ST > 1200"
#     ],
#     coupled=False
# )